# Access a Database with Python - Iris Dataset

The Iris dataset is a popular dataset especially in the Machine Learning community, it is a set of features of 50  Iris flowers and their classification into 3 species.
It is often used to introduce classification Machine Learning algorithms.

First let's download the dataset in `SQLite` format from Kaggle:

<https://www.kaggle.com/uciml/iris/>

Download `database.sqlite` and save it in the `data/iris` folder.

<p><img   src="https://upload.wikimedia.org/wikipedia/commons/4/49/Iris_germanica_%28Purple_bearded_Iris%29%2C_Wakehurst_Place%2C_UK_-_Diliff.jpg" alt="Iris germanica (Purple bearded Iris), Wakehurst Place, UK - Diliff.jpg" height="145" width="114"></p>

<p><br> From <a href="https://commons.wikimedia.org/wiki/File:Iris_germanica_(Purple_bearded_Iris),_Wakehurst_Place,_UK_-_Diliff.jpg#/media/File:Iris_germanica_(Purple_bearded_Iris),_Wakehurst_Place,_UK_-_Diliff.jpg">Wikimedia</a>, by <a href="//commons.wikimedia.org/wiki/User:Diliff" title="User:Diliff">Diliff</a> - <span class="int-own-work" lang="en">Own work</span>, <a href="http://creativecommons.org/licenses/by-sa/3.0" title="Creative Commons Attribution-Share Alike 3.0">CC BY-SA 3.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=33037509">Link</a></p>

First let's check that the sqlite database is available and display an error message if the file is not available (`assert` checks if the expression is `True`, otherwise throws `AssertionError` with the error message string provided):

In [25]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
import os
data_iris_folder_content = os.listdir("data/iris")

In [3]:
error_message = "Error: sqlite file not available, check instructions above to download it"
assert "database.sqlite" in data_iris_folder_content, error_message

## Access the Database with the sqlite3 Package

We can use the `sqlite3` package from the Python standard library to connect to the `sqlite` database:

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('data/iris/database.sqlite')
conn

In [3]:
cursor = conn.cursor()
cursor

In [4]:
type(cursor)

sqlite3.Cursor

A `sqlite3.Cursor` object is our interface to the database, mostly throught the `execute` method that allows to run any `SQL` query on our database.

First of all we can get a list of all the tables saved into the database, this is done by reading the column `name` from the `sqlite_master` metadata table with:

    SELECT name FROM sqlite_master
    
The output of the `execute` method is an iterator that can be used in a `for` loop to print the value of each row.

In [8]:
for row in cursor.execute("SELECT name FROM sqlite_master"):
    print(row)

('Iris',)
('grade',)


a shortcut to directly execute the query and gather the results is the `fetchall` method:

In [9]:
cursor.execute("SELECT name FROM sqlite_master").fetchall()

[('Iris',), ('grade',)]

**Notice**: this way of finding the available tables in a database is specific to `sqlite`, other databases like `MySQL` or `PostgreSQL` have different syntax.

Then we can execute standard `SQL` query on the database, `SQL` is a language designed to interact with data stored in a relational database. It has a standard specification, therefore the commands below work on any database.

If you need to connect to another database, you would use another package instead of `sqlite3`, for example:

* [MySQL Connector](https://dev.mysql.com/doc/connector-python/en/) for MySQL
* [Psycopg](http://initd.org/psycopg/docs/install.html) for PostgreSQL
* [pymssql](http://pymssql.org/en/stable/) for Microsoft MS SQL

then you would connect to the database using specific host, port and authentication credentials but then you could execute the same exact `SQL` statements.

Let's take a look for example at the first 3 rows in the Iris table:

In [14]:
sample_data = cursor.execute("SELECT * FROM Iris LIMIT 20").fetchall()

In [19]:
print(type(sample_data))
sample_data

<class 'list'>


[(1, 5.1, 3.5, 1.4, 0.2, 'Iris-setosa'),
 (2, 4.9, 3, 1.4, 0.2, 'Iris-setosa'),
 (3, 4.7, 3.2, 1.3, 0.2, 'Iris-setosa'),
 (4, 4.6, 3.1, 1.5, 0.2, 'Iris-setosa'),
 (5, 5, 3.6, 1.4, 0.2, 'Iris-setosa'),
 (6, 5.4, 3.9, 1.7, 0.4, 'Iris-setosa'),
 (7, 4.6, 3.4, 1.4, 0.3, 'Iris-setosa'),
 (8, 5, 3.4, 1.5, 0.2, 'Iris-setosa'),
 (9, 4.4, 2.9, 1.4, 0.2, 'Iris-setosa'),
 (10, 4.9, 3.1, 1.5, 0.1, 'Iris-setosa'),
 (11, 5.4, 3.7, 1.5, 0.2, 'Iris-setosa'),
 (12, 4.8, 3.4, 1.6, 0.2, 'Iris-setosa'),
 (13, 4.8, 3, 1.4, 0.1, 'Iris-setosa'),
 (14, 4.3, 3, 1.1, 0.1, 'Iris-setosa'),
 (15, 5.8, 4, 1.2, 0.2, 'Iris-setosa'),
 (16, 5.7, 4.4, 1.5, 0.4, 'Iris-setosa'),
 (17, 5.4, 3.9, 1.3, 0.4, 'Iris-setosa'),
 (18, 5.1, 3.5, 1.4, 0.3, 'Iris-setosa'),
 (19, 5.7, 3.8, 1.7, 0.3, 'Iris-setosa'),
 (20, 5.1, 3.8, 1.5, 0.3, 'Iris-setosa')]

In [16]:
# cursor.description - This read-only attribute provides the column names of the last query.
# https://docs.python.org/2/library/sqlite3.html#sqlite3.Cursor.description
[row[0] for row in cursor.description] 

['Id',
 'SepalLengthCm',
 'SepalWidthCm',
 'PetalLengthCm',
 'PetalWidthCm',
 'Species']

It is evident that the interface provided by `sqlite3` is low-level, for data exploration purposes we would like to directly import data into a more user friendly library like `pandas`.

## Import data from a database to `pandas`

In [10]:
import pandas as pd

In [11]:
iris_data = pd.read_sql_query("SELECT * FROM Iris", conn)

In [12]:
iris_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [23]:
iris_data.dtypes

Id                 int64
SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
Species           object
dtype: object

`pandas.read_sql_query` takes a `SQL` query and a connection object and imports the data into a `DataFrame`, also keeping the same data types of the database columns. `pandas` provides a lot of the same functionality of `SQL` with a more user-friendly interface.

However, `sqlite3` is extremely useful for downsizing data **before** importing them in `pandas`.

For example you might have 1 TB of data in a table stored in a database on a server machine. You are interested in working on a subset of the data based on some criterion, unfortunately it would be impossible to first load data into `pandas` and then filter them, therefore we should tell the database to perform the filtering and just load into `pandas` the downsized dataset.

In [27]:
iris_setosa_data = pd.read_sql_query("SELECT * FROM Iris WHERE Species == 'Iris-setosa'", conn)
iris_setosa_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [25]:
# iris_setosa_data
print(iris_setosa_data.shape)
print(iris_data.shape)


(50, 6)
(150, 6)


### Creating a TABLE :


In [26]:
# Creating another table in the database: 
# this will throw a error if it already exists.
# cursor.execute('CREATE TABLE grade(id int, name text, score int)')

In [2]:
import sqlite3
conn = sqlite3.connect('data/iris/database.sqlite')
cursor = conn.cursor()
cursor.execute('SELECT name FROM sqlite_master').fetchall()

[('Iris',), ('grade',)]

In [3]:
cursor.execute('DELETE FROM grade')
conn.commit()

In [4]:
# Add values to database One Row at a time
cursor.execute("INSERT INTO grade(id, name, score) VALUES(101, 'veena', 98)")
cursor.execute("INSERT INTO grade(id, name, score) VALUES(103, 'veena', 101)")


In [5]:
# Adding multiple rows at a time
data = [
    (104,'seema', 94),
    (106, 'naina', 87),
    (392, 'hassd', 34),
    (234, 'Naina', 455)
]
query_stmt = "INSERT INTO grade VALUES(?,?,?)"
cursor.executemany(query_stmt, data)

In [6]:
# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [8]:
conn = sqlite3.connect('data/iris/database.sqlite')
cursor = conn.cursor()
cursor.execute("SELECT * FROM grade").fetchall()

[(101, 'veena', 98),
 (103, 'veena', 101),
 (104, 'seema', 94),
 (106, 'naina', 87),
 (392, 'hassd', 34),
 (234, 'Naina', 455)]

In [11]:
pd.read_sql_query("SELECT * FROM grade", conn)
conn.close()

# MongoDB

1. MongoDB is a cross-platform document-oriented database program. Classified as a NoSQL database program, MongoDB uses JSON-like documents with optional schemas. The data is stored as Key-Value pairs.
2. Sql vs MongoDB database:
    * A SQL DB server consists of multiple DBs and each DB will have multiple tables. Each tables in turn will have multiple Rows.
    * A mongoDB server will also consists of multiple DBs and each DB will have multiple __Collections__ (Here Collections is used instrad of Tables). Each collection will have __Records/Documents__ (equivalent to rows). Each record or document is key value pay.

1. Download the latest "Mongo DB community server" version and install MongoDB. Installing is recomended, so you will not have to run the mongo db server everytime to use it.
2. Open cmd and go to 'C:\Program Files\MongoDB\Server\4.2\bin' and enter mongo to confirm that we have installed it.
3. Mongo Compass comes installed wiht MongoDB which is a gui for it.


In [71]:
import pymongo # package which enables us to connect to mongodb
dbConn = pymongo.MongoClient('mongodb://localhost:27017') # connect to local running mongodb instance
dbConn

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [72]:
print(dbConn.list_database_names())
db = dbConn['DemoDB'] # connect to DB named 'NamedDB' present in DB.
print(db.list_collection_names())
collection = db['student'] # obtain the collection(table) name

['DemoDB', 'admin', 'config', 'local']
['student']


In [73]:
# single insert
entry1 = {
    'name': 'Veena Kalburgi',
    'rollNo': 12345,
    'email': 'veena.kal@gmail.com',
    'major': 'Computer Science',
    'yob': 1988
}
collection.insert_one(entry1) # insert a record, mydict is a dict with key-value pair

In [74]:
# multiple insert
entry2 = [
    {
    'name': 'Seema Mehar',
    'rollNo': 56789,
    'email': 'seema.meh@gmail.com',
    'major': 'Electronics',
    'yob': 1978
    },
    {
    'name': 'Maan Khurana',
    'rollNo': 26544,
    'email': 'maan.geet@gmail.com',
    'major': 'Computer Science',
    'yob': 1985
    },
    {
    'name': 'Geet Handa',
    'rollNo': 5487,
    'email': 'geet.maan@gmail.com',
    'major': 'Stylish',
    'yob': 1989
    }
    
]
collection.insert_many(entry2)

In [75]:
# retieving all the records from collection
records = collection.find({}) 
for record in records:
    print(record)

{'_id': ObjectId('5ecefe3389f4add9ff8759f7'), 'name': 'Veena Kalburgi', 'rollNo': 12345, 'email': 'veena.kal@gmail.com', 'major': 'Computer Science', 'yob': 1988}
{'_id': ObjectId('5ecefe3389f4add9ff8759f8'), 'name': 'Seema Mehar', 'rollNo': 56789, 'email': 'seema.meh@gmail.com', 'major': 'Electronics', 'yob': 1978}
{'_id': ObjectId('5ecefe3389f4add9ff8759f9'), 'name': 'Maan Khurana', 'rollNo': 26544, 'email': 'maan.geet@gmail.com', 'major': 'Computer Science', 'yob': 1985}
{'_id': ObjectId('5ecefe3389f4add9ff8759fa'), 'name': 'Geet Handa', 'rollNo': 5487, 'email': 'geet.maan@gmail.com', 'major': 'Stylish', 'yob': 1989}


In [76]:
# retrieveing some columns
students = collection.find({}, {'name'}).limit(5) # retrieveing two columns
# The second parameter in find() specifies which columns to choose
for student in students:
#     print(type(student)) # will give a dict
    print(student['name'])

Veena Kalburgi
Seema Mehar
Maan Khurana
Geet Handa


In [84]:
# query for data
my_query = {'yob':1988}
res = collection.find(my_query, {'name', 'yob'})

for r in res:
    pp.pprint(r)

{   '_id': ObjectId('5ecefe3389f4add9ff8759f7'),
    'name': 'Veena Kalburgi',
    'yob': 1988}


In [78]:
# querey for data with condition
my_query = {'yob': {'$gt': 1980}} # yob is greater than 1980
res = collection.find(my_query).sort('yob', -1) # sort by 'yob' in descending order (-1)

for r in res:
    pp.pprint(r)

{   '_id': ObjectId('5ecefe3389f4add9ff8759fa'),
    'email': 'geet.maan@gmail.com',
    'major': 'Stylish',
    'name': 'Geet Handa',
    'rollNo': 5487,
    'yob': 1989}
{   '_id': ObjectId('5ecefe3389f4add9ff8759f7'),
    'email': 'veena.kal@gmail.com',
    'major': 'Computer Science',
    'name': 'Veena Kalburgi',
    'rollNo': 12345,
    'yob': 1988}
{   '_id': ObjectId('5ecefe3389f4add9ff8759f9'),
    'email': 'maan.geet@gmail.com',
    'major': 'Computer Science',
    'name': 'Maan Khurana',
    'rollNo': 26544,
    'yob': 1985}


In [79]:
# delete records
del_stud = {'name': 'Seema Mehar'}
collection.delete_one(del_stud)
for rec in collection.find({}):
    pp.pprint(rec)

{   '_id': ObjectId('5ecefe3389f4add9ff8759f7'),
    'email': 'veena.kal@gmail.com',
    'major': 'Computer Science',
    'name': 'Veena Kalburgi',
    'rollNo': 12345,
    'yob': 1988}
{   '_id': ObjectId('5ecefe3389f4add9ff8759f9'),
    'email': 'maan.geet@gmail.com',
    'major': 'Computer Science',
    'name': 'Maan Khurana',
    'rollNo': 26544,
    'yob': 1985}
{   '_id': ObjectId('5ecefe3389f4add9ff8759fa'),
    'email': 'geet.maan@gmail.com',
    'major': 'Stylish',
    'name': 'Geet Handa',
    'rollNo': 5487,
    'yob': 1989}


In [80]:
# for dropping a collection
print(db.list_collection_names())
# db.drop_collection('test')

['student']


In [81]:
print(dbConn.list_database_names())
print(db.list_collection_names())

['DemoDB', 'admin', 'config', 'local']
['student']


In [82]:
update_criteria={'major':'Stylish'}
new_values= { "$set": { "major": "Fashion Designer" } }
collection.update_many(update_criteria, new_values)
for rec in collection.find({}):
    pp.pprint(rec)

{   '_id': ObjectId('5ecefe3389f4add9ff8759f7'),
    'email': 'veena.kal@gmail.com',
    'major': 'Computer Science',
    'name': 'Veena Kalburgi',
    'rollNo': 12345,
    'yob': 1988}
{   '_id': ObjectId('5ecefe3389f4add9ff8759f9'),
    'email': 'maan.geet@gmail.com',
    'major': 'Computer Science',
    'name': 'Maan Khurana',
    'rollNo': 26544,
    'yob': 1985}
{   '_id': ObjectId('5ecefe3389f4add9ff8759fa'),
    'email': 'geet.maan@gmail.com',
    'major': 'Fashion Designer',
    'name': 'Geet Handa',
    'rollNo': 5487,
    'yob': 1989}


In [83]:
# dbConn.close()